# Adding external features to our dataset:

In this noteboook what we are going to do is add features that, according to the factory affect demand. Now, it is important to remember that our data corresponds to the orders made by the store to the factory, and not sales, hence features like weather or events may explain sales behaviour, however if the stores are not taking that information into account, then our time-series of orders would not be affected by these features.

Nonetheless, the factory has suggested us that stores take into account these features. Lets see...

The features that we will be adding are:
- Football matches: currently we are only taking into account La Liga matches, however it would be idea to extend it to Champions, Euro Cup and World Cup (and to other sports such as tennis, and olympic games).

- Festivities (public holidays).


Other datasets that would be interesting to add would be:

- Weather: Historical predictions for the following 7 days (if accurate), not historical weather.


In [1]:
import pandas as pd
import numpy as np

In [2]:
# basic paths
base_path = '../data/_auxiliary_data/'
matches_file_name = 'MATCHES_2013-2019.xlsx'
festivities_file_name = 'festivities_calendar_Madrid_2013-2020.csv'
input_path = "../data/03_processed/" + "1_filtered_transactions_clean.csv"
filtered_file_name="c1-filtered_transactions.csv"
sep=";"
exit_path = "../data/03_processed/" + "2_time_series.csv"


## Matches

We have several teams in the Comunidad de Madrid, that will predictably have a different impact in our sales.

Based on their impact, we agreed to divide the teams in four categories:

* Real Madrid
* Ath Madrid
* Other from Comunidad de Madrid
* Rest

Based on this categorization, we have 8 different basic scenarios:

* RM -AM
* RM - Other Madrid
* RM - Rest
* AM - Other Madrid
* AM - Rest
* Other Madrid - Other Madrid
* Other Madrid - Rest
* Rest - Rest


Lets now create manually a label enconder to take into consideration these 8 scenarios:


In [3]:
# Read matches
matches = pd.read_excel(base_path+matches_file_name,parse_dates = ['Date'] )
matches.columns =  [col.lower() for col in matches.columns]
matches.head()

,div,date,hometeam,awayteam,home,away,matches_madrid
0,SP1,2012-08-18,Celta,Malaga,0,0,0
1,SP1,2012-08-18,Mallorca,Espanol,0,0,0
2,SP1,2012-08-18,Sevilla,Getafe,0,1,1
3,SP1,2012-08-19,Ath Bilbao,Betis,0,0,0
4,SP1,2012-08-19,Barcelona,Sociedad,0,0,0


Lets verify that the team names are normalized (one team - one name):


In [4]:
matches["hometeam"].value_counts()

Sevilla        152
Barcelona      152
Real Madrid    152
Ath Madrid     152
Valencia       152
Espanol        152
Sociedad       152
Celta          152
Ath Bilbao     152
Malaga         133
Betis          133
Villarreal     133
Levante        114
Granada        114
La Coruna      114
Eibar          114
Getafe         114
Vallecano       95
Osasuna         76
Las Palmas      76
Leganes         76
Alaves          76
Valladolid      57
Sp Gijon        57
Elche           38
Girona          38
Almeria         38
Zaragoza        19
Huesca          19
Mallorca        19
Cordoba         19
Name: hometeam, dtype: int64

In [5]:
matches["awayteam"].value_counts()

Sevilla        152
Barcelona      152
Real Madrid    152
Ath Madrid     152
Valencia       152
Espanol        152
Sociedad       152
Celta          152
Ath Bilbao     152
Malaga         133
Betis          133
Villarreal     133
Levante        114
Granada        114
La Coruna      114
Eibar          114
Getafe         114
Vallecano       95
Osasuna         76
Las Palmas      76
Leganes         76
Alaves          76
Valladolid      57
Sp Gijon        57
Elche           38
Girona          38
Almeria         38
Zaragoza        19
Huesca          19
Mallorca        19
Cordoba         19
Name: awayteam, dtype: int64

In [6]:
def funct_label_encod(team):   
    """Given a team name, returns 4 if real madrid, 2 if atletico, or 0 if other """
    if team=='Real Madrid':
        return 4
    elif team=='Ath Madrid':
        return 2
    else:
        return 0

# Calculates the lable encoder described before:
matches['label_encoder']=(matches['home'] +
                                   matches['away'] +
                                   matches['hometeam'].apply(funct_label_encod) + 
                                   matches['awayteam'].apply(funct_label_encod))

In [7]:
matches[matches['label_encoder']>0].shape

(1086, 8)

Woops, we actually didn't take into account that we needed to aggregate the data in order to have a unique value per day. 

Nonetheless, what we can do to fix the issue of having too many lables is using as weights. Lets modify the previous function to give the same weight to Real Madrid and Atletico de Madrid.

Note: in order to prove that both Real Madrid and Atletico de Madrid have the same impact, we would need to do a full analysis, however due to time constrains we can consider them as equal.

In [8]:
def weight_of_team (team):   
    """Given a team name, returns 3 if real madrid or Atletico """
    if team=='Real Madrid' or team=='Ath Madrid' :
        return 3
    else:
        return 0

# Calculates the lable encoder described before:
matches['weight']=(matches['home'] +
                                   matches['away'] +
                                   matches['hometeam'].apply(funct_label_encod) + 
                                   matches['awayteam'].apply(funct_label_encod))

In [9]:
df_matches = matches.groupby(['date'])['weight'].sum().reset_index()

Lets now filter to only keep values greater than 0 (for performance purpose when merging)

In [10]:
# Lets only include days with at least one point of weight:
df_matches = df_matches[df_matches['weight']>0][['date','weight']]

In [11]:
df_matches

,date,weight
0,2012-08-18,1
1,2012-08-19,8
2,2012-08-20,1
3,2012-08-25,1
4,2012-08-26,6
...,...,...
910,2019-05-04,4
911,2019-05-05,6
912,2019-05-12,11
913,2019-05-18,6


In [12]:
#Testing variables

validator_num_matches = df_matches[(df_matches['date']<= '2019-09-30') & (df_matches['date']>= '2008-01-01')].shape[0]
validator_num_matches

612

## Festivities

Now lets do a analogues thing for the public holidays. However, instead of setting weights, lets have a two categorical labels:

- 1 if day is public holiday
- 0 if not. 

In [13]:
df_public_holidays = pd.read_csv(base_path+festivities_file_name, encoding = "ISO-8859-1", sep = ';', parse_dates = ['Dia'])
df_public_holidays = df_public_holidays.loc[df_public_holidays['laborable / festivo / domingo festivo']=='festivo','Dia'].to_frame()
df_public_holidays['festivo'] = 1
df_public_holidays.head()

,Dia,festivo
0,2013-01-01,1
6,2013-07-01,1
76,2013-03-18,1
86,2013-03-28,1
87,2013-03-29,1


In [14]:
#Testing variables

validator_num_holidays = df_public_holidays[(df_public_holidays['Dia']<= '2019-09-30') & (df_public_holidays['Dia']>= '2008-01-01')].shape[0]
validator_num_holidays

102

## Adding it to the transactions dataframe

In [15]:
# Reading the file
df = pd.read_csv(input_path, sep=sep, parse_dates = ['order_date'])

In [16]:
# Grouping by product and date:
df = df.groupby(['order_date','product'])['units_ordered'].sum().reset_index()
df.shape

(42910, 3)

In [17]:
# Testing Variables
original_shape = df.shape[0]

### Adding Public holidays

In [18]:
df_holidays = pd.merge(df, df_public_holidays, how = 'left', left_on='order_date', right_on='Dia')

In [19]:
df_holidays[df_holidays['festivo']==1]['order_date'].unique().shape

(102,)

In [20]:
df_holidays.head()

,order_date,product,units_ordered,Dia,festivo
0,2008-01-01,baguette,86.0,NaT,NaN
1,2008-01-01,croissant petit,178.0,NaT,NaN
2,2008-01-01,croissant simple,137.0,NaT,NaN
3,2008-01-01,milhojas frambuesa,12.0,NaT,NaN
4,2008-01-01,mousse tres chocolates,2.0,NaT,NaN


In [21]:
df_holidays.shape

(42910, 5)

In [22]:
(df_holidays['festivo']>0).sum()

1020

In [23]:
#Tests

if df_holidays.shape[0] != original_shape:
    test_holiday = (0,"ERROR - original dataset != holidays dataset")
elif validator_num_holidays != df_holidays.loc[df_holidays['festivo']>0, "order_date"].unique().shape[0]:
    test_holiday = (0,"ERROR - holiday numbers != holidays numbers in merged dataset")
else:
    test_holiday = (1,"All good")
print(test_holiday)

(1, 'All good')


### Adding matches

In [24]:
df_holidays_matches = pd.merge(df_holidays, df_matches, how = 'left', left_on='order_date', right_on='date')

In [25]:
df_holidays_matches.head()

,order_date,product,units_ordered,Dia,festivo,date,weight
0,2008-01-01,baguette,86.0,NaT,NaN,NaT,NaN
1,2008-01-01,croissant petit,178.0,NaT,NaN,NaT,NaN
2,2008-01-01,croissant simple,137.0,NaT,NaN,NaT,NaN
3,2008-01-01,milhojas frambuesa,12.0,NaT,NaN,NaT,NaN
4,2008-01-01,mousse tres chocolates,2.0,NaT,NaN,NaT,NaN


In [26]:
df_holidays_matches = df_holidays_matches[['order_date', 'product', 'units_ordered', 'festivo','weight']]

In [27]:
#Tests

if df_holidays_matches.shape[0] != original_shape:
    test_matches = (0,"ERROR - original dataset != matches dataset")
elif validator_num_matches != df_holidays_matches.loc[df_holidays_matches['weight']>0, "order_date"].unique().shape[0]:
    test_matches = (0,"ERROR - holiday numbers != holidays numbers in merged dataset")
else:
    test_matches = (1,"All good")
print(test_matches)

(1, 'All good')


# Cleaning NaN created

In this case NaN correspond to 0, so lets replace them

In [28]:
df_holidays_matches.fillna(0, inplace = True)

In [29]:
df_holidays_matches.head()

,order_date,product,units_ordered,festivo,weight
0,2008-01-01,baguette,86.0,0.0,0.0
1,2008-01-01,croissant petit,178.0,0.0,0.0
2,2008-01-01,croissant simple,137.0,0.0,0.0
3,2008-01-01,milhojas frambuesa,12.0,0.0,0.0
4,2008-01-01,mousse tres chocolates,2.0,0.0,0.0


# Tests results & Save to file

In [30]:
if (test_matches[0] and test_holiday[0]):
    print("All the tests have been passed. Saving to file")
    df_holidays_matches.to_csv(exit_path, sep=sep, index=False)
else:
    print("Something went wrong, please reviwe errors")
    print(test_matches)
    print(test_holiday)

All the tests have been passed. Saving to file
